In [1]:
LOAD_MODELS = False
RUN_NUM = 1

In [2]:
!cp -r ../input/updated-code-3-7/* /kaggle/working/

In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 8.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 53.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 37.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.9/466.9 kB 25.3 MB/s eta 0:00:00
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=464219 sha256=b60bbf59a91199c2d801f5f00bc165d87489867df1b023947938504bbead4681
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
  Created wheel for jupyter-contrib-nbextensions: filename=jupyter_contrib_nbextensions-0.7.0-py2.py3-none-any.whl size=23428781 sha256=67bc91f17646b81814d328ee017e5b6a1b7b5705b2f3c149adad6e708cbea478
  Stored in directory: /root/.cache/pip/wheels/ea/cc/7d/99

In [3]:
!pip install lightfm tqdm optuna ipykernel matplotlib implicit

  Obtaining dependency information for implicit from https://files.pythonhosted.org/packages/cd/cc/deac70cae8cc32c9885d0cd73bc66e1b3cbea36ae7080b8c83995eaf5322/implicit-0.7.2-cp310-cp310-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 49.8 MB/s eta 0:00:0000:0100:01


In [4]:
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
/opt/conda/lib/python3.10/site-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /kaggle/working/Recommenders/MatrixFactorization/Cython/MatrixFactorization_Cython_Epoch.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... PASS

Compiling [2/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/co

# Imports

In [4]:
import time
import optuna
import numpy as np
import pandas as pd
import scipy.sparse as sps
from scipy.sparse import vstack, csr_matrix
from scipy.stats import loguniform
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k
import matplotlib.pyplot as plt
from multiprocessing import cpu_count
from tqdm import tqdm
import os

seed = 69
np.random.seed(seed)

In [5]:
from Recommenders.BaseRecommender import BaseRecommender
from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender
from Recommenders.NonPersonalizedRecommender import TopPop

#---- CF
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

#---- Matrix Factorization
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#---- CF w/ ML
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
from implicit.als import AlternatingLeastSquares

#---- Gradient Boosting
import xgboost as xgb
from xgboost import XGBRanker

#---- Others
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.Recommender_utils import check_matrix
from implicit.utils import check_csr
from Recommenders.DataIO import DataIO

# Run Versioning (For Model + Split saving)

In [6]:
BASE_DIR = "Runs/"

In [7]:
def get_next_folder_number(base_dir):
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    existing_folders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]
    existing_numbers = [int(f[1:]) for f in existing_folders if f.startswith('#') and f[1:].isdigit()]

    next_number = 1
    if existing_numbers:
        next_number = max(existing_numbers) + 1

    return f"#{next_number}"

def create_versioned_folder(base_dir):
    folder_name = get_next_folder_number(base_dir)
    new_folder_path = os.path.join(base_dir, folder_name)
    os.makedirs(new_folder_path)
    return new_folder_path

In [8]:
run_folder = create_versioned_folder(BASE_DIR)
print(f"New folder created: {run_folder}")

New folder created: Runs/#2


# Data Import and Preprocessing

In [9]:
# Import training data
URM_path = "../input/data-books/data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path,
                                header=0,
                                dtype={0:int, 1:int, 2:int},
                                engine='python')

URM_all_dataframe.columns = ["user_id", "item_id", "interaction"]

In [10]:
# Import target users
target_path = "../input/data-books/data_target_users_test.csv"
target_dataframe= pd.read_csv(filepath_or_buffer=target_path,
                                header=0,
                                dtype={0:int},
                                engine='python')
target_dataframe.columns = ["user_id"]
target_dataframe

,user_id
0,1
1,2
2,3
3,4
4,5
...,...
10877,13020
10878,13021
10879,13022
10880,13023


In [11]:
def preprocess_data(ratings: pd.DataFrame):
    unique_users = ratings.user_id.unique()
    unique_items = ratings.item_id.unique()

    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()

    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)

    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

    ratings = pd.merge(left=ratings,
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")

    ratings = pd.merge(left=ratings,
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

    return ratings

In [12]:
# Call preprocess data function
ratings = preprocess_data(URM_all_dataframe)

12638 1 13024
22222 1 22347


## Conversion to Sparse Matrix

In [13]:
URM = sps.coo_matrix((ratings.interaction.values, (ratings.mapped_user_id.values, ratings.mapped_item_id.values)))

## Data Splits

In [14]:
def split_urm_in_k_folds(URM_all, k):
    from Data_manager.IncrementalSparseMatrix import IncrementalSparseMatrix
    num_users, num_items = URM_all.shape

    URM_train = sps.coo_matrix(URM_all)

    indices_for_sampling = np.arange(0, URM_all.nnz, dtype=int)
    np.random.shuffle(indices_for_sampling)

    indices_for_sampling = np.array_split(indices_for_sampling, k)
    np.random.shuffle(indices_for_sampling)
    k_URM = []
    
    for i in range(k):
        URM_builder = IncrementalSparseMatrix(n_rows=num_users, n_cols=num_items, auto_create_col_mapper=False, auto_create_row_mapper=False)
        URM_builder.add_data_lists(URM_train.row[indices_for_sampling[i]],
                                             URM_train.col[indices_for_sampling[i]],
                                             URM_train.data[indices_for_sampling[i]])

        k_URM.append(sps.csr_matrix(URM_builder.get_SparseMatrix()))

    #verify that the sum of the URM_train_matrices is equal to the original URM
    k_URM_sum = sps.csr_matrix((num_users, num_items))
    for URM_train_matrices_single in k_URM:
        k_URM_sum += URM_train_matrices_single
    
    assert k_URM_sum.nnz == URM_all.nnz, "split_train_in_k_percentage_global_sample: URM_train_matrices_sum doesn't match URM_all"


    return k_URM

In [15]:
#urm_train, urm_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
#urm_train, urm_validation = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)

#evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
#evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

In [16]:
k = 10
splits = split_urm_in_k_folds(URM, k)

In [17]:
urm_train = sum(splits[:7])
urm_val = sum(splits[7:9])
urm_test = splits[9]
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 3623 (28.7%) Users that have less than 1 test interactions


In [18]:
sps.save_npz(f"{run_folder}/URM_train.npz", urm_train)
sps.save_npz(f"{run_folder}/URM_val.npz", urm_val)
sps.save_npz(f"{run_folder}/URM_test.npz", urm_test)

# Load Data (Pre-split)

In [7]:
run_folder = f"{BASE_DIR}#{RUN_NUM}" # Runs/#1/

In [8]:
urm_train = sps.load_npz(f"{run_folder}/URM_train.npz")
urm_val = sps.load_npz(f"{run_folder}/URM_val.npz")
urm_test = sps.load_npz(f"{run_folder}/URM_test.npz")
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 3623 (28.7%) Users that have less than 1 test interactions


# Custom XGRecommender Class

In [9]:
class Recommender():
    def __init__(self, recommender_name, recommender_class, params_dict):
        self.recommender_name = recommender_name
        self.recommender_class = recommender_class
        self.params_dict = params_dict
        self.recommender_instance = None
    
    def fit(self, URM):
        self.recommender_instance = self.recommender_class(URM)
        self.recommender_instance.fit(**self.params_dict)
        
    def load(self, URM, run_dir, metadata=""):
        models_path = f"{run_dir}/Models/"
        file_name = f"{self.recommender_name}_{metadata}.zip"
        self.recommender_instance = self.recommender_class(URM)
        if os.path.exists(models_path+file_name):
            self.recommender_instance.load_model(models_path, file_name)
        else:
            print(f"Model {file_name} not found at {models_path}, re-training...")
            self.fit(URM)
            self.save(run_dir, metadata)
        
    def save(self, run_dir, metadata):
        self.recommender_instance.save_model(f"{run_dir}/Models/", f"{self.recommender_name}_{metadata}.zip")
        
    def __copy__(self):
        self.normalizeArgs()
        return Recommender(self.recommender_name, self.recommender_class, self.params_dict)
    
    def normalizeArgs(self):
        if not hasattr(self, "recommender_name"):
            self.recommender_name = "Unknown"
        if not hasattr(self, "recommender_class"):
            raise ValueError("Recommender has no attribute recommender_class. Aborting copy operation.")
        if not hasattr(self, "params_dict"):
            raise ValueError("Recommender has no attribute params_dict. Aborting copy operation.")

In [10]:
class XGRecommender(BaseRecommender):
    global run_folder
    """ XGRecommender
    Gradient boosting recommender

    """

    RECOMMENDER_NAME = "XGRecommender"

    def __init__(self, URM_train, URM_val, recommenders, load_models=False):
        super(XGRecommender, self).__init__(URM_train)
        self.URM_train = sps.csr_matrix(URM_train)
        self.URM_val = sps.csr_matrix(URM_val)
        self.recommenders = recommenders
        self.recommenders_train = []
        self.recommenders_val = []
        if load_models:
            for rid in tqdm(range(len(recommenders))):
                recommender_train = recommenders[rid]
                recommender_val = recommender_train.__copy__()

                recommender_train.load(self.URM_train, run_folder, "train")
                self.recommenders_train.append(recommender_train)

                recommender_val.load(self.URM_train, run_folder, "val")
                self.recommenders_val.append(recommender_val)
        else:
            for rid in tqdm(range(len(recommenders))):
                recommender_train = recommenders[rid]
                recommender_val = recommender_train.__copy__()

                recommender_train.fit(self.URM_train)
                recommender_train.save(run_folder, "train")
                self.recommenders_train.append(recommender_train)

                recommender_val.fit(self.URM_train + self.URM_val)
                recommender_val.save(run_folder, "val")
                self.recommenders_val.append(recommender_val)
            
    def recommender_lookup(self, name, collection):
        result = None
        for recommender in collection:
            if recommender.recommender_name.lower() == name:
                result = recommender
                break
        if result == None:
            print(f"No recommender {name} found")
        return result
            
    def get_recommender_train(self, name):
        return self.recommender_lookup(name, self.recommenders_train)
    
    def get_recommender_train_val(self, name):
        return self.recommender_lookup(name, self.recommenders_val)

    def fit(self, training_dataframe,
            n_estimators = 50,
            learning_rate = 1e-1,
            reg_alpha = 1e-1,
            reg_lambda = 1e-1,
            max_depth = 5,
            max_leaves = 0,
            grow_policy = "depthwise",
            objective = "pairwise",
            booster = "gbtree",
            use_user_profile = False,
            random_seed = None,
            tree_method = "hist",  # Supported tree methods are `gpu_hist`, `approx`, and `hist`.
            colsample_bytree = 1,
            gamma = 0,
            min_child_weight = 0
           ):
        training_dataframe = training_dataframe.set_index('UserID')

        for user_id in tqdm(range(self.n_users)):  
            for recommender in self.recommenders_train:
                rec_instance = recommender.recommender_instance
                rec_label = recommender.recommender_name

                item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()

                all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)

                training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list] 

        training_dataframe = training_dataframe.reset_index()
        training_dataframe = training_dataframe.rename(columns = {"index": "UserID"})
        
        self.XGB_model = XGBRanker(objective='rank:{}'.format(objective),
                          n_estimators = int(n_estimators),
                          random_state = random_seed,
                          learning_rate = learning_rate,
                          reg_alpha = reg_alpha,
                          reg_lambda = reg_lambda,
                          max_depth = int(max_depth),
                          max_leaves = int(max_leaves),
                          grow_policy = grow_policy,
                          verbosity = 0, # 2 if self.verbose else 0,
                          booster = booster,
                          enable_categorical = True,
                          tree_method = tree_method,  # Supported tree methods are `gpu_hist`, `approx`, and `hist`.
                          colsample_bytree = colsample_bytree,
                          gamma = gamma,
                          min_child_weight = min_child_weight
                      )
        groups = training_dataframe.groupby("UserID").size().values
        
        y_train = training_dataframe["Label"]
        X_train = training_dataframe.drop(columns=["Label"])

        #in order to prevent the algorithm to split user groups according to whether their ID is of a certain value
        X_train["UserID"] = X_train["UserID"].astype("category")
        X_train["ItemID"] = X_train["ItemID"].astype("category")
        
        print("Training...")
        self.XGB_model.fit(
          X_train,
          y_train,
          group=groups,
          verbose=True)
        
        #xgb.plot_importance(self.XGB_model, importance_type='weight', title='Weight (Frequence)')
        
    def prepare_validation_dataframe(self, base_validation_dataframe):
        print("Setting-up val_df...")
        self.validation_dataframe = base_validation_dataframe
        validation_dataframe_tmp = self.validation_dataframe.set_index('UserID')

        for user_id in tqdm(range(self.n_users)):  
            for recommender in self.recommenders_val:
                rec_instance = recommender.recommender_instance
                rec_label = recommender.recommender_name

                item_list = validation_dataframe_tmp.loc[user_id, "ItemID"].values.tolist()

                all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)

                validation_dataframe_tmp.loc[user_id, rec_label] = all_item_scores[0, item_list] 

        validation_dataframe_tmp = validation_dataframe_tmp.reset_index()
        self.validation_dataframe = validation_dataframe_tmp.rename(columns = {"index": "UserID"})
        self.prepare_rankings()
        
    def prepare_rankings(self):
        print("Predicting...")
        predictions = self.XGB_model.predict(self.validation_dataframe)
        self.validation_dataframe['rating_xgb'] = pd.Series(predictions, index=self.validation_dataframe.index)
        self.validation_dataframe = self.validation_dataframe.sort_values(['UserID','rating_xgb'], ascending=[True, False])
        print("Evaluating...")

    def _compute_item_score(self, user_id_array, items_to_compute):
        item_scores = - np.ones((len(user_id_array), self.n_items)) * np.inf
        for user_index, user_id in enumerate(user_id_array):
            item_scores[user_index, self.validation_dataframe.loc[self.validation_dataframe['UserID'] == user_id].ItemID] = self.validation_dataframe.loc[self.validation_dataframe['UserID'] == user_id].rating_xgb.values
        return item_scores
    
    def show_importance(self):
        print("Importance: ")
        plot_importance(self.XGB_model, importance_type='weight', title='Weight (Frequence)')

# Other Custom Classes

In [11]:
class FastIALSRecommender(BaseMatrixFactorizationRecommender):
    RECOMMENDER_NAME = "FastIALSRecommender"

    AVAILABLE_CONFIDENCE_SCALING = ["linear", "log"]
    
    def __init__(self, URM_train, verbose=True):
        super().__init__(URM_train, verbose=verbose)
        
    def fit(self,
            factors=20,
            regularization=1e-3,
            iterations=100,
            calculate_training_loss=False,
            num_threads=0,
            confidence_scaling='linear',
            alpha=1.0,
            epsilon=0,
            #---- Do not change
            use_native=True,
            use_cg=True,
            use_gpu=True):
        if confidence_scaling not in self.AVAILABLE_CONFIDENCE_SCALING:
           raise ValueError("Value for 'confidence_scaling' not recognized. Acceptable values are {}, provided was '{}'".format(self.AVAILABLE_CONFIDENCE_SCALING, confidence_scaling))

        self.alpha = alpha
        self.epsilon = epsilon
        self.num_factors = factors
        self.reg = regularization
        
        self.USER_factors = self._init_factors(self.n_users, False)  # don't need values, will compute them
        self.ITEM_factors = self._init_factors(self.n_items)
        
        self.recommender = AlternatingLeastSquares(factors=factors, regularization=regularization,
                                                        use_native=use_native, use_cg=use_cg, use_gpu=use_gpu,
                                                        iterations=iterations,
                                                        calculate_training_loss=calculate_training_loss,
                                                        num_threads=num_threads)
        
        self._build_confidence_matrix(confidence_scaling)
        self.recommender.fit(self.C, show_progress=self.verbose)
        
        self.USER_factors = self.recommender.user_factors.to_numpy()
        self.ITEM_factors = self.recommender.item_factors.to_numpy()
        
    
    def _linear_scaling_confidence(self):

        C = check_matrix(self.URM_train, format="csr", dtype = np.float32)
        C.data = 1.0 + self.alpha*C.data

        return C

    def _log_scaling_confidence(self):

        C = check_matrix(self.URM_train, format="csr", dtype = np.float32)
        C.data = 1.0 + self.alpha * np.log(1.0 + C.data / self.epsilon)

        return C
    
    def _build_confidence_matrix(self, confidence_scaling):

        if confidence_scaling == 'linear':
            self.C = self._linear_scaling_confidence()
        else:
            self.C = self._log_scaling_confidence()

        self.C_csc= check_matrix(self.C.copy(), format="csc", dtype = np.float32)
        
    def carrefour(self, numpy_array):
        rows, cols = numpy_array.nonzero()
        data_values = numpy_array[rows, cols]
        return sps.coo_matrix((data_values, (rows, cols)), shape=numpy_array.shape).tocsr()
    
    def _init_factors(self, num_factors, assign_values=True):

        if assign_values:
            return self.num_factors**-0.5*np.random.random_sample((num_factors, self.num_factors))

        else:
            return np.empty((num_factors, self.num_factors))

In [12]:
class LightFMCFRecommender(BaseRecommender):
    """LightFMCFRecommender"""

    RECOMMENDER_NAME = "LightFMCFRecommender"

    def __init__(self, URM_train):
        super(LightFMCFRecommender, self).__init__(URM_train)
    
    
    def fit(self, epochs = 200, user_alpha=1e-6, item_alpha = 1e-6, n_factors = 10, n_threads = 4, max_sampled=3, loss='warp', learning_schedule='adagrad'):
        
        # Let's fit a WARP model
        self.lightFM_model = LightFM(loss=loss,
                                     user_alpha=user_alpha,
                                     item_alpha=item_alpha,
                                     no_components=n_factors,
                                     max_sampled=max_sampled,
                                     learning_schedule=learning_schedule)

        self.lightFM_model = self.lightFM_model.fit(self.URM_train, 
                                       epochs=epochs,
                                       num_threads=n_threads,
                                       verbose=True)


    def _compute_item_score(self, user_id_array, items_to_compute = None):
        
        # Create a single (n_items, ) array with the item score, then copy it for every user
        items_to_compute = np.arange(self.n_items)
        
        item_scores = - np.ones((len(user_id_array), self.n_items)) * np.inf

        for user_index, user_id in enumerate(user_id_array):
            item_scores[user_index] = self.lightFM_model.predict(int(user_id), 
                                                                 items_to_compute)

        return item_scores
    
    def save_model(self, folder_path, file_name = None):

        if file_name is None:
            file_name = self.RECOMMENDER_NAME

        self._print("Saving model in file '{}'".format(folder_path + file_name))

        data_dict_to_save = {
                            "item_embeddings": self.lightFM_model.item_embeddings,
                            "item_embedding_gradients": self.lightFM_model.item_embedding_gradients,
                            "item_embedding_momentum": self.lightFM_model.item_embedding_momentum,
                            "item_biases": self.lightFM_model.item_biases,
                            "item_bias_gradients": self.lightFM_model.item_bias_gradients,
                            "item_bias_momentum": self.lightFM_model.item_bias_momentum,
                            "user_embeddings": self.lightFM_model.user_embeddings,
                            "user_embedding_gradients": self.lightFM_model.user_embedding_gradients,
                            "user_embedding_momentum": self.lightFM_model.user_embedding_momentum,
                            "user_biases": self.lightFM_model.user_biases,
                            "user_bias_gradients": self.lightFM_model.user_bias_gradients,
                            "user_bias_momentum": self.lightFM_model.user_bias_momentum,
                            }


        dataIO = DataIO(folder_path=folder_path)
        dataIO.save_data(file_name=file_name, data_dict_to_save = data_dict_to_save)

        self._print("Saving complete")
        
        
    def load_model(self, folder_path, file_name = None):

        if file_name is None:
            file_name = self.RECOMMENDER_NAME

        self._print("Loading model from file '{}'".format(folder_path + file_name))

        dataIO = DataIO(folder_path=folder_path)
        data_dict = dataIO.load_data(file_name=file_name)

        self.lightFM_model = LightFM()

        for attrib_name in data_dict.keys():
             self.lightFM_model.__setattr__(attrib_name, data_dict[attrib_name])

        self._print("Loading complete")

# Main Model params (MAP-Optimized)

In [13]:
SLIM_params = {
 'l1_ratio': 0.04324773367371399,
 'alpha': 0.001220701931267383,
 'topK': 971
}

In [14]:
RP3Beta_params = {
 'alpha': 0.3302958327908062,
 'beta': 0.14271386569051958,
 'topK': 29,
 'normalize_similarity': True
}

In [15]:
alpha_slim_rp3beta = 0.5356582511094522

# Model params (RECALL-Optimized)

In [16]:
ItemKNN_params = {
    'topK': 90,
    'shrink': 1168,
    'similarity': 'jaccard',
    'normalize': False,
    'feature_weighting': 'BM25'
}

In [17]:
UserKNN_params = {
    'topK': 331,
    'shrink': 315,
    'similarity': 'cosine',
    'normalize': True,
    'feature_weighting': 'TF-IDF'
}

In [18]:
IALS_params = {
    'confidence_scaling': 'log',
    'epsilon': 0.0028130992083797435,
    'factors': 302,
    'regularization': 0.009832686029765005,
    'alpha': 1.7031521523379438
}

In [19]:
RP3Beta_params_rec = {
    'alpha': 0.3535736692428584,
    'beta': 0.23632895406841348,
    'topK': 60,
    'normalize_similarity': True
}

In [20]:
SLIM_params_rec = {
    'l1_ratio': 0.01998720297988901,
    'alpha': 0.0032805805487669745,
    'topK': 698
}

In [21]:
LightFM_params = {
                  'n_factors': 482,
                  'max_sampled': 5,
                  'user_alpha': 0.00023989649900734266,
                  'item_alpha': 9.740651135253414e-05
                 }

# Testing Custom XGBoost Class

In [22]:
cutoff = 30
top_pop_cutoff = 10

In [23]:
def build_train_dataframe(rec, urm_train, urm_val, top_pop=None):
    global cutoff
    n_users, n_items = urm_train.shape

    # Initialize lists to store the data
    user_ids = []
    item_ids = []
    top_pop_flags = []
    
    for user_id in tqdm(range(n_users)):    
        # Get recommendations from the main recommender
        recommendations = rec.recommend(user_id, cutoff=cutoff)
        user_ids.extend([user_id] * len(recommendations))
        item_ids.extend(recommendations)
        top_pop_flags.extend([False] * len(recommendations))  # False for main recommender
        
        if top_pop is not None:
            # Get top-pop recommendations
            top_pop_recommendations = top_pop.recommend(user_id, cutoff=cutoff)
            user_ids.extend([user_id] * len(top_pop_recommendations))
            item_ids.extend(top_pop_recommendations)
            top_pop_flags.extend([True] * len(top_pop_recommendations))  # True for top-pop recommendations
    
    # Create the training dataframe from the lists
    training_dataframe = pd.DataFrame({
        'UserID': user_ids,
        'ItemID': item_ids,
        'TopPop': top_pop_flags
    })
    
    urm_validation_coo = sps.coo_matrix(urm_val)
    correct_recommendations = pd.DataFrame({
        "UserID": urm_validation_coo.row, 
        "ItemID": urm_validation_coo.col
    })
    # Merge to find out which recommendations are correct
    training_dataframe = pd.merge(training_dataframe, correct_recommendations, 
                                  on=['UserID', 'ItemID'], 
                                  how='left', indicator='Exist')
    # Create the label column
    training_dataframe["Label"] = training_dataframe["Exist"] == "both"
    training_dataframe.drop(columns=['Exist'], inplace=True)
    
    #adding item popularity to the df
    item_popularity = np.ediff1d(sps.csc_matrix(urm_train).indptr)
    training_dataframe['item_popularity'] = item_popularity[training_dataframe["ItemID"].values.astype(int)]

    #adding user profile length to the df
    user_popularity = np.ediff1d(sps.csr_matrix(urm_train).indptr)
    training_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]
    
    return training_dataframe

In [24]:
def build_val_dataframe(rec, urm, top_pop=None):
    global cutoff
    # Initialize lists to store the data
    user_ids = []
    item_ids = []
    top_pop_flags = []

    for user_id in tqdm(range(urm.shape[0])):    
        # Get recommendations from the main recommender
        recommendations = rec.recommend(user_id, cutoff=cutoff)
        user_ids.extend([user_id] * len(recommendations))
        item_ids.extend(recommendations)
        top_pop_flags.extend([False] * len(recommendations))  # False for main recommender
        
        if top_pop is not None:
            # Get top-pop recommendations
            top_pop_recommendations = top_pop.recommend(user_id, cutoff=cutoff)
            user_ids.extend([user_id] * len(top_pop_recommendations))
            item_ids.extend(top_pop_recommendations)
            top_pop_flags.extend([True] * len(top_pop_recommendations))  # True for top-pop recommendations
    
    # Create the validation dataframe from the lists
    val_dataframe = pd.DataFrame({
        'UserID': user_ids,
        'ItemID': item_ids,
        'TopPop': top_pop_flags
    })
    
    #adding item popularity to the df
    item_popularity = np.ediff1d(sps.csc_matrix(urm).indptr)
    val_dataframe['item_popularity'] = item_popularity[training_dataframe["ItemID"].values.astype(int)]

    #adding user profile length to the df
    user_popularity = np.ediff1d(sps.csr_matrix(urm).indptr)
    val_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]
    
    return val_dataframe

## Building main model (SLIM+RP3Beta)

### Load Base Model

In [25]:
base_model = "SLIM_RP3Beta"

In [26]:
base_model_train_path = f"{run_folder}/Models/"
base_model_filename = f"base_{base_model}_train.zip"

In [27]:
slim_rp3beta = ItemKNNCustomSimilarityRecommender(urm_train)
slim_rp3beta.load_model(base_model_train_path, base_model_filename)

ItemKNNCustomSimilarityRecommender: URM Detected 408 ( 3.2%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 230 ( 1.0%) items with no interactions.
ItemKNNCustomSimilarityRecommender: Loading model from file 'Runs/#4/Models/base_SLIM_RP3Beta_train.zip'
ItemKNNCustomSimilarityRecommender: Loading complete


### Train Base Model

In [23]:
rp3beta = RP3betaRecommender(urm_train)
rp3beta.fit(**RP3Beta_params)

RP3betaRecommender: URM Detected 408 ( 3.2%) users with no interactions.
RP3betaRecommender: URM Detected 230 ( 1.0%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 3960.48 column/sec. Elapsed time 5.61 sec


In [24]:
slim = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train)
slim.fit(**SLIM_params, workers = int(cpu_count()))

SLIMElasticNetRecommender: URM Detected 408 ( 3.2%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 230 ( 1.0%) items with no interactions.


100%|█████████▉| 22216/22222 [15:14<00:00, 24.29it/s]


In [25]:
new_similarity = (1 - alpha_slim_rp3beta) * slim.W_sparse + alpha_slim_rp3beta * rp3beta.W_sparse
    
slim_rp3beta = ItemKNNCustomSimilarityRecommender(urm_train)
slim_rp3beta.fit(new_similarity)

ItemKNNCustomSimilarityRecommender: URM Detected 408 ( 3.2%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 230 ( 1.0%) items with no interactions.


## Building other models for construction of DF

In [28]:
top_pop = TopPop(urm_train)
top_pop.fit()

TopPopRecommender: URM Detected 408 ( 3.2%) users with no interactions.
TopPopRecommender: URM Detected 230 ( 1.0%) items with no interactions.


In [29]:
training_dataframe = build_train_dataframe(slim_rp3beta, urm_train, urm_val, top_pop)

100%|██████████| 12638/12638 [00:16<00:00, 772.98it/s]


In [30]:
itemKNN_rec = Recommender('ItemKNN', ItemKNNCFRecommender, ItemKNN_params)
userKNN_rec = Recommender('UserKNN', UserKNNCFRecommender, UserKNN_params)
rp3beta_rec = Recommender('RP3Beta', RP3betaRecommender, RP3Beta_params_rec)
slim_rec = Recommender("SLIM", MultiThreadSLIM_SLIMElasticNetRecommender, SLIM_params_rec)
IALS = Recommender('IALS', FastIALSRecommender, IALS_params)
lightFM = Recommender('lightFM', LightFMCFRecommender, LightFM_params)

In [31]:
recommenders = [itemKNN_rec, userKNN_rec, rp3beta_rec, slim_rec, IALS, lightFM]

In [32]:
xg_rec = XGRecommender(urm_train, urm_val, recommenders, LOAD_MODELS)

XGRecommender: URM Detected 408 ( 3.2%) users with no interactions.
XGRecommender: URM Detected 230 ( 1.0%) items with no interactions.


  0%|          | 0/6 [00:00<?, ?it/s]

ItemKNNCFRecommender: URM Detected 408 ( 3.2%) users with no interactions.
ItemKNNCFRecommender: URM Detected 230 ( 1.0%) items with no interactions.
ItemKNNCFRecommender: Loading model from file 'Runs/#4/Models/ItemKNN_train.zip'
ItemKNNCFRecommender: Loading complete
ItemKNNCFRecommender: URM Detected 408 ( 3.2%) users with no interactions.
ItemKNNCFRecommender: URM Detected 230 ( 1.0%) items with no interactions.
ItemKNNCFRecommender: Loading model from file 'Runs/#4/Models/ItemKNN_val.zip'


 17%|█▋        | 1/6 [00:00<00:02,  1.98it/s]

ItemKNNCFRecommender: Loading complete
UserKNNCFRecommender: URM Detected 408 ( 3.2%) users with no interactions.
UserKNNCFRecommender: URM Detected 230 ( 1.0%) items with no interactions.
UserKNNCFRecommender: Loading model from file 'Runs/#4/Models/UserKNN_train.zip'
UserKNNCFRecommender: Loading complete
UserKNNCFRecommender: URM Detected 408 ( 3.2%) users with no interactions.
UserKNNCFRecommender: URM Detected 230 ( 1.0%) items with no interactions.
UserKNNCFRecommender: Loading model from file 'Runs/#4/Models/UserKNN_val.zip'


...

SLIMElasticNetRecommender: Loading complete
FastIALSRecommender: URM Detected 408 ( 3.2%) users with no interactions.
FastIALSRecommender: URM Detected 230 ( 1.0%) items with no interactions.
FastIALSRecommender: Loading model from file 'Runs/#4/Models/IALS_train.zip'
FastIALSRecommender: Loading complete
FastIALSRecommender: URM Detected 408 ( 3.2%) users with no interactions.
FastIALSRecommender: URM Detected 230 ( 1.0%) items with no interactions.
FastIALSRecommender: Loading model from file 'Runs/#4/Models/IALS_val.zip'


 83%|████████▎ | 5/6 [00:04<00:01,  1.10s/it]

FastIALSRecommender: Loading complete
LightFMCFRecommender: URM Detected 408 ( 3.2%) users with no interactions.
LightFMCFRecommender: URM Detected 230 ( 1.0%) items with no interactions.
LightFMCFRecommender: Loading model from file 'Runs/#4/Models/lightFM_train.zip'
LightFMCFRecommender: Loading complete
LightFMCFRecommender: URM Detected 408 ( 3.2%) users with no interactions.
LightFMCFRecommender: URM Detected 230 ( 1.0%) items with no interactions.
LightFMCFRecommender: Loading model from file 'Runs/#4/Models/lightFM_val.zip'


100%|██████████| 6/6 [00:10<00:00,  1.71s/it]

LightFMCFRecommender: Loading complete


## Preparing for Validation

### Load Base Model

In [33]:
base_model_val_path = f"{run_folder}/Models/"
base_model_val_file = f"base_{base_model}_val.zip"

In [34]:
slim_rp3beta_val = ItemKNNCustomSimilarityRecommender(urm_train + urm_val)
slim_rp3beta_val.load_model(base_model_val_path, base_model_val_file)

ItemKNNCustomSimilarityRecommender: URM Detected 129 ( 1.0%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 35 ( 0.2%) items with no interactions.
ItemKNNCustomSimilarityRecommender: Loading model from file 'Runs/#4/Models/base_SLIM_RP3Beta_val.zip'
ItemKNNCustomSimilarityRecommender: Loading complete


### Train Base Model

In [28]:
rp3beta_val = RP3betaRecommender(urm_train + urm_val)
rp3beta_val.fit(**RP3Beta_params)

RP3betaRecommender: URM Detected 129 ( 1.0%) users with no interactions.
RP3betaRecommender: URM Detected 35 ( 0.2%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 3349.49 column/sec. Elapsed time 6.63 sec


In [29]:
slim_val = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train + urm_val)
slim_val.fit(**SLIM_params, workers = int(cpu_count()))

SLIMElasticNetRecommender: URM Detected 129 ( 1.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 35 ( 0.2%) items with no interactions.


100%|█████████▉| 22216/22222 [15:50<00:00, 23.36it/s]


In [30]:
new_similarity = (1 - alpha_slim_rp3beta) * slim.W_sparse + alpha_slim_rp3beta * rp3beta.W_sparse
    
slim_rp3beta_val = ItemKNNCustomSimilarityRecommender(urm_train + urm_val)
slim_rp3beta_val.fit(new_similarity)

ItemKNNCustomSimilarityRecommender: URM Detected 129 ( 1.0%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 35 ( 0.2%) items with no interactions.


## Build Validation DF

In [35]:
top_pop_val = TopPop(urm_train + urm_val)
top_pop_val.fit()

TopPopRecommender: URM Detected 129 ( 1.0%) users with no interactions.
TopPopRecommender: URM Detected 35 ( 0.2%) items with no interactions.


In [36]:
val_dataframe = build_val_dataframe(slim_rp3beta_val, urm_val, top_pop_val)

100%|██████████| 12638/12638 [00:17<00:00, 723.72it/s]


# Save Base Models

In [31]:
base_model = "SLIM_RP3Beta"

In [34]:
run_folder+"/Models"

'Runs/#4/Models'

In [35]:
slim_rp3beta.save_model(f"{run_folder}/Models/", f"base_{base_model}_train.zip")
slim_rp3beta_val.save_model(f"{run_folder}/Models/", f"base_{base_model}_val.zip")

ItemKNNCustomSimilarityRecommender: Saving model in file 'Runs/#4/Models/base_SLIM_RP3Beta_train.zip'
ItemKNNCustomSimilarityRecommender: Saving complete
ItemKNNCustomSimilarityRecommender: Saving model in file 'Runs/#4/Models/base_SLIM_RP3Beta_val.zip'
ItemKNNCustomSimilarityRecommender: Saving complete


# Optuna optimization

In [37]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame(columns=["result"])

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        # Create a DataFrame from the current trial's results
        trial_df = pd.DataFrame([hyperparam_dict])

        # Use concat instead of append
        self.results_df = pd.concat([self.results_df, trial_df], ignore_index=True)

In [40]:
def objective_function_xgranker(optuna_trial):
    learning_rate = 1e-2
    grow_policy = "depthwise"
    objective = "pairwise"
    booster = "gbtree"
    use_user_profile = False
    random_seed = None
    tree_method = "hist"
    
    n_estimators = optuna_trial.suggest_int("n_estimators", 100, 3500)
    max_depth = optuna_trial.suggest_int("max_depth", 2, 10)
    max_leaves = optuna_trial.suggest_int("max_leaves", 0, 10)
    reg_alpha = optuna_trial.suggest_float("reg_alpha", 1e-1, 300)
    reg_lambda = optuna_trial.suggest_float("reg_lambda", 1e-4, 2)
    #colsample_bytree = optuna_trial.suggest_float("colsample_bytree", 0.2, 1)
    #gamma = optuna_trial.suggest_float("gamma", 0, 10)
    min_child_weight = optuna_trial.suggest_float("min_child_weight", 0, 10)
    
    xg_rec.fit(training_dataframe,
                  learning_rate = learning_rate,
                  reg_alpha = reg_alpha,
                  reg_lambda = reg_lambda,
                  grow_policy = grow_policy,
                  objective = objective,
                  booster = booster,
                  use_user_profile = use_user_profile,
                  random_seed = random_seed,
                  tree_method = tree_method,
                  n_estimators = n_estimators,
                  max_depth = max_depth,
                  max_leaves = max_leaves,
                  #colsample_bytree = colsample_bytree,
                  #gamma = gamma,
                  min_child_weight = min_child_weight
              )
    xg_rec.prepare_validation_dataframe(val_dataframe)

    result_df, _ = evaluator_test.evaluateRecommender(xg_rec)

    return result_df.loc[10]["MAP"]

In [ ]:
optuna_study_xgranker = optuna.create_study(direction="maximize")

save_results_xgranker = SaveResults()

optuna_study_xgranker.optimize(objective_function_xgranker,
                      callbacks=[save_results_xgranker],
                      n_trials = 300)

[I 2024-01-08 14:01:01,778] A new study created in memory with name: no-name-a4f34ea7-189e-4f6c-8e05-57c6b107502a
 45%|████▌     | 5727/12638 [05:44<06:50, 16.84it/s]

In [ ]:
{'n_estimators': 2689, 'max_depth': 4, 'max_leaves': 0, 'reg_alpha': 23.888878705922945, 'reg_lambda': 0.31381253860663455} - value: 0.027594569016366064